In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from datetime import datetime, timedelta
warnings.filterwarnings("ignore")

three_factor_file_name = "Data/05_df_ff_info.csv"
three_factor_df = pd.read_csv(three_factor_file_name, index_col = 0)

fundamental_file_name = "Data/funda.csv"
fundamental_df = pd.read_csv(fundamental_file_name, index_col = 0)

monthly_stocks_file_name = "Data/monthlystocks.csv"
monthly_stocks_df = pd.read_csv(monthly_stocks_file_name, index_col=0)



In [2]:
#Make your masks for filtering the fundamentals data


mask_1 = (fundamental_df['indfmt'] == 'INDL')
mask_2= (fundamental_df['curcd'] == 'USD')
mask_3 = ((fundamental_df['exchg'] >= 11) & (fundamental_df['exchg'] <= 19))
mask_4 = (fundamental_df['fic'] == 'USA')
mask_5 = ((fundamental_df['sic'] < 6000) | (fundamental_df['sic'] > 6999))

fundamental_update = fundamental_df[mask_1&mask_2&mask_3&mask_4&mask_5].copy()
fundamental_update

,LPERMNO,datadate,fyear,indfmt,consol,popsrc,datafmt,curcd,csho,xrd,exchg,costat,fic,mkvalt,prcc_f,sic
GVKEY,,,,,,,,,,,,,,,,
1000,25881,12/31/1975,1975.0,INDL,C,D,STD,USD,2.098,NaN,12,I,USA,NaN,4.375,3089.0
1000,25881,12/31/1976,1976.0,INDL,C,D,STD,USD,2.207,NaN,12,I,USA,NaN,5.750,3089.0
1000,25881,12/31/1977,1977.0,INDL,C,D,STD,USD,2.226,NaN,12,I,USA,NaN,9.250,3089.0
1001,10015,12/31/1983,1983.0,INDL,C,D,STD,USD,3.568,0.000,14,I,USA,NaN,7.250,5812.0
1001,10015,12/31/1984,1984.0,INDL,C,D,STD,USD,3.568,NaN,14,I,USA,NaN,3.750,5812.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345920,20194,12/31/2021,2021.0,INDL,C,D,STD,USD,44.618,NaN,14,A,USA,1262.2432,28.290,3524.0
345980,20333,12/31/2020,2020.0,INDL,C,D,STD,USD,587.000,222.000,14,A,USA,10706.8800,18.240,5961.0
345980,20333,12/31/2021,2021.0,INDL,C,D,STD,USD,658.000,208.000,14,A,USA,2046.3800,3.110,5961.0


In [3]:

monthly_stocks_df = monthly_stocks_df[pd.to_numeric(monthly_stocks_df['RET'], errors='coerce').notnull()]


monthly_stocks_df['date'] = pd.to_datetime(monthly_stocks_df['date'])

monthly_stocks_df['RET'] = pd.to_numeric(monthly_stocks_df['RET'])

monthly_stocks_df = monthly_stocks_df[(monthly_stocks_df['RET'] > -100)]

monthly_stocks_df = monthly_stocks_df[(monthly_stocks_df['SHROUT']) > 0]

monthly_stocks_df['PRC'] = abs(monthly_stocks_df['PRC'])

monthly_stocks_df['PRC'] = monthly_stocks_df['PRC'].replace({0:np.nan})

monthly_stocks_df = monthly_stocks_df.dropna(subset = ['SHROUT', 'PRC'])

monthly_stocks_df['mkcap'] = monthly_stocks_df['PRC']*monthly_stocks_df['SHROUT']

monthly_stocks_df

,date,PRC,VOL,RET,SHROUT,mkcap
PERMNO,,,,,,
10000,1986-02-28,3.25000,828.0,-0.257143,3680.0,1.196000e+04
10000,1986-03-31,4.43750,1078.0,0.365385,3680.0,1.633000e+04
10000,1986-04-30,4.00000,957.0,-0.098592,3793.0,1.517200e+04
10000,1986-05-30,3.10938,1074.0,-0.222656,3793.0,1.179388e+04
10000,1986-06-30,3.09375,1069.0,-0.005025,3793.0,1.173459e+04
...,...,...,...,...,...,...
93436,2021-08-31,735.71997,3812156.0,0.070605,1001767.0,7.370200e+08
93436,2021-09-30,775.47998,3889228.0,0.054042,1004000.0,7.785819e+08
93436,2021-10-29,1114.00000,5263955.0,0.436530,1004265.0,1.118751e+09


In [5]:
fundamental_update['datadate'] = pd.to_datetime(fundamental_update['datadate'])

fundamental_update['date_90'] = fundamental_update['datadate'] + pd.DateOffset(days = 85) + pd.tseries.offsets.MonthEnd(0)
fundamental_update['date_90_1'] = fundamental_update['date_90'] + pd.DateOffset(years=1)

fundamental_update['year'] = fundamental_update['datadate'].dt.year

fundamental_update['xrd'] = fundamental_update['xrd'].fillna(0)

fundamental_update = fundamental_update.sort_values(by=['datadate','LPERMNO']).reset_index(drop=True)

fundamental_update['RDC'] = fundamental_update['xrd'] + \
                    0.8*fundamental_update.groupby('LPERMNO')['xrd'].shift(1) + \
                    0.6*fundamental_update.groupby('LPERMNO')['xrd'].shift(2) + \
                    0.4*fundamental_update.groupby('LPERMNO')['xrd'].shift(3) + \
                    0.2*fundamental_update.groupby('LPERMNO')['xrd'].shift(4)



In [6]:
fundamental_update

,LPERMNO,datadate,fyear,indfmt,consol,popsrc,datafmt,curcd,csho,xrd,exchg,costat,fic,mkvalt,prcc_f,sic,date_90,date_90_1,year,RDC
0,10102,1975-12-31,1975.0,INDL,C,D,STD,USD,11.411,12.992,11,I,USA,NaN,17.625,2810.0,1976-03-31,1977-03-31,1975,NaN
1,10137,1975-12-31,1975.0,INDL,C,D,STD,USD,27.292,0.000,11,I,USA,NaN,18.500,4911.0,1976-03-31,1977-03-31,1975,NaN
2,10145,1975-12-31,1975.0,INDL,C,D,STD,USD,27.909,35.181,14,A,USA,NaN,33.250,9997.0,1976-03-31,1977-03-31,1975,NaN
3,10153,1975-12-31,1975.0,INDL,C,D,STD,USD,10.702,40.100,11,I,USA,NaN,12.000,1381.0,1976-03-31,1977-03-31,1975,NaN
4,10161,1975-12-31,1975.0,INDL,C,D,STD,USD,31.080,18.110,11,I,USA,NaN,47.250,3334.0,1976-03-31,1977-03-31,1975,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181462,93371,2021-12-31,2021.0,INDL,C,D,STD,USD,38.086,13.133,14,A,USA,1.732913e+02,4.550,2836.0,2022-03-31,2023-03-31,2021,42.8924
181463,93372,2021-12-31,2021.0,INDL,C,D,STD,USD,22.981,10.152,11,A,USA,8.976379e+02,39.060,3531.0,2022-03-31,2023-03-31,2021,20.7738
181464,93423,2021-12-31,2021.0,INDL,C,D,STD,USD,86.163,0.000,11,A,USA,3.668820e+03,42.580,7996.0,2022-03-31,2023-03-31,2021,0.0000
181465,93426,2021-12-31,2021.0,INDL,C,D,STD,USD,13.626,17.200,11,A,USA,5.057971e+02,37.120,3674.0,2022-03-31,2023-03-31,2021,41.6000
